<a href="https://colab.research.google.com/github/CyrillGK/NLP/blob/main/Stepik_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import datetime
import copy

import matplotlib.pyplot as plt
import collections

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from google.colab import drive
drive.mount("drive")

In [ ]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install import-ipynb
import import_ipynb
import tools

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=dbae9d143b1279098fa25f8d19c2001f555822ffe5f167db64e071e510388780
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
importing Jupyter notebook from tools.ipynb


https://github.com/Samsung-IT-Academy/stepik-dl-nlp/blob/master/task1_20newsgroups.ipynb

## Классификация новостных сайтов

In [ ]:
train_source = fetch_20newsgroups(subset='train')
test_source = fetch_20newsgroups(subset='test')
len(train_source['data']), len(test_source['data'])

(11314, 7532)

### Линейная модель

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC

In [ ]:
res = pd.DataFrame(columns=['CE_train', 'F1_train', 'CE_test', 'F1_test'])
pipe = Pipeline((
    ('vect', TfidfVectorizer()),
    ('cls', LinearSVC())
))
pipe.fit(train_source['data'], train_source['target']);
train_pred = pipe.predict(train_source['data'])
test_pred = pipe.predict(test_source['data'])

#train_ce = float(F.cross_entropy(torch.from_numpy(train_pred), torch.from_numpy(train_source['target'])))
train_f1 = f1_score(train_source['target'], train_pred, average='weighted')
#test_ce = float(F.cross_entropy(torch.from_numpy(test_pred), torch.from_numpy(test_source['target'])))
test_f1 = f1_score(test_source['target'], test_pred, average='weighted')
train_f1, test_f1
#res = res.append({'CE_train':train_ce, 'F1_train':train_f1, 'CE_test':test_ce, 'F1_test':test_f1}, ignore_index=True)
#res

(0.9990279582521118, 0.851876062902485)

### Линейный слой на TF-IDF

In [ ]:
import tools

In [ ]:
tfidf = TfidfVectorizer(max_features=1500)
X_train = tfidf.fit_transform(train_source['data'], train_source['target'])
X_test = tfidf.transform(test_source['data'])

In [ ]:
class SparseFeaturesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        cur_features = torch.from_numpy(self.features[idx].toarray()[0]).float()
        cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
        return cur_features, cur_label

In [ ]:
train_dataset = SparseFeaturesDataset(X_train, train_source['target'])
test_dataset = SparseFeaturesDataset(X_test, test_source['target'])

In [ ]:
class NLPNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(NLPNet, self).__init__()
        self.fc1 = torch.nn.Linear(1500, n_hidden_neurons)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 20)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = NLPNet(750)
#model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

In [ ]:
def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device='cpu', early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000, max_batches_per_epoch_val=1000,
                    shuffle_train=True, collate_fn=None):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle_train, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    for epoch_i in range(epoch_n):
        epoch_start = datetime.datetime.now()
        print('Эпоха {}'.format(epoch_i))

        model.train()
        mean_train_loss = 0
        train_batches_n = 0
        for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
            if batch_i > max_batches_per_epoch_train:
                break

            batch_x = tools.copy_data_to_device(batch_x, device)
            batch_y = tools.copy_data_to_device(batch_y, device)

            pred = model(batch_x)
            loss = criterion(pred, batch_y)

            model.zero_grad()
            loss.backward()

            optimizer.step()

            mean_train_loss += float(loss)
            train_batches_n += 1

        mean_train_loss /= train_batches_n
        print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n, (datetime.datetime.now() - epoch_start).total_seconds()))
        print('Среднее значение функции потерь на обучении', mean_train_loss)

        model.eval()
        mean_val_loss = 0
        val_batches_n = 0

        with torch.no_grad():
            for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                if batch_i > max_batches_per_epoch_val:
                    break

                batch_x = tools.copy_data_to_device(batch_x, device)
                batch_y = tools.copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)

                mean_val_loss += float(loss)
                val_batches_n += 1

        mean_val_loss /= val_batches_n
        print('Среднее значение функции потерь на валидации', mean_val_loss)

        if mean_val_loss < best_val_loss:
            best_epoch_i = epoch_i
            best_val_loss = mean_val_loss
            best_model = copy.deepcopy(model)
            print('Новая лучшая модель!')
        elif epoch_i - best_epoch_i > early_stopping_patience:
            print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                early_stopping_patience))
            break

        lr_scheduler.step(mean_val_loss)
        print()

    return best_val_loss, best_model

In [ ]:
_, best_model = train_eval_loop(model=model,
                                train_dataset=train_dataset,
                                val_dataset=test_dataset,
                                criterion=F.cross_entropy,
                                lr=1e-1,
                                epoch_n=30,
                                batch_size=32,
                                l2_reg_alpha=1e-5)

Эпоха 0
Эпоха: 354 итераций, 4.93 сек
Среднее значение функции потерь на обучении 0.6297133692492873
Среднее значение функции потерь на валидации 1.4333481806314599
Новая лучшая модель!

Эпоха 1
Эпоха: 354 итераций, 4.74 сек
Среднее значение функции потерь на обучении 0.4961384821704215
Среднее значение функции потерь на валидации 1.554447521092528

Эпоха 2
Эпоха: 354 итераций, 4.80 сек
Среднее значение функции потерь на обучении 0.48384858800445574
Среднее значение функции потерь на валидации 1.558987007050191

Эпоха 3
Эпоха: 354 итераций, 4.98 сек
Среднее значение функции потерь на обучении 0.46802780195270893
Среднее значение функции потерь на валидации 1.5466190515938452

Эпоха 4
Эпоха: 354 итераций, 4.93 сек
Среднее значение функции потерь на обучении 0.43811458911178475
Среднее значение функции потерь на валидации 1.6619163648051731

Эпоха 5
Эпоха: 354 итераций, 4.73 сек
Среднее значение функции потерь на обучении 0.4497166555480095
Среднее значение функции потерь на валидации 1.

In [ ]:
train_pred = tools.predict_with_model(best_model, train_dataset)
train_loss = F.cross_entropy(torch.from_numpy(train_pred), torch.from_numpy(train_source['target']).long())

print('Среднее значение функции потерь на обучении', float(train_loss))
print('Доля верных ответов', accuracy_score(train_source['target'], train_pred.argmax(-1)))
print()

test_pred = tools.predict_with_model(best_model, test_dataset)
test_loss = F.cross_entropy(torch.from_numpy(test_pred), torch.from_numpy(test_source['target']).long())

print('Среднее значение функции потерь на валидации', float(test_loss))
print('Доля верных ответов', accuracy_score(test_source['target'], test_pred.argmax(-1)))

Среднее значение функции потерь на обучении 0.3254581093788147
Доля верных ответов 0.8986211773024572

Среднее значение функции потерь на валидации 1.4330660104751587
Доля верных ответов 0.6311736590546999


### Embedding

In [ ]:
from torchtext.vocab import Vocab
from collections import Counter

In [ ]:
tokenize = TfidfVectorizer().build_analyzer()
vocab = Vocab(
    Counter(
        tokenize(' '.join(train_source['data']))
    )
, min_freq=1, max_size=20000)

In [ ]:
text_pipeline = lambda x: [vocab.freqs[token] for token in tokenize(x)]

In [ ]:
def collate_batch(batch):
    label_list = []
    for (_text, _label) in batch:
         label_list.append(_label)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    vectorized_seqs = [text_pipeline(seq[0]) for seq in batch]
    seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))
    seq_tensor = torch.autograd.Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
    for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]
    return (seq_tensor.to(device), seq_lengths), label_list.to(device)

#### Линейный слой с Embedding

In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return (text_list.to(device), offsets.to(device)), label_list.to(device)

In [ ]:
class LinearEmbNet(torch.nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(LinearEmbNet, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, emb_dim)
        self.fc2 = torch.nn.Linear(emb_dim, 20)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
model = LinearEmbNet(len(vocab), 1000)

In [ ]:
_, best_model = train_eval_loop(model=model,
                                train_dataset=train_dataset,
                                val_dataset=test_dataset,
                                criterion=F.cross_entropy,
                                lr=1e-1,
                                epoch_n=30,
                                batch_size=32,
                                l2_reg_alpha=1e-5,
                                collate_fn=collate_fn)

### Сверточный слой

In [ ]:
# TODO

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = torch.cat((pooled_0, pooled_1, pooled_2), dim=1)

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

### Рекуррентный слой

In [ ]:
# TODO

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, n_layers):
        super(NewModel, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(input_size=embed_dim, hidden_size=hidden_size, num_layers=n_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(2*hidden_size, 98)    
        
        def init_hidden(self, batch_size):
          weight = next(self.parameters()).data
          hidden = weight.new(2*self.n_layers, batch_size, self.hidden_size).zero_().to(device)
        return hidden    
        
        def forward(self, data, h):
          embedded = self.embedding(data[0])
          x, h = self.gru(embedded, h)
          #print(x[:,-1].shape)
        return self.fc(x[:,-1]), h

In [ ]:
def train_eval_l(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32, device='cpu', early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000, max_batches_per_epoch_val=1000, shuffle_train=True, collate_fn=None):    
    device = torch.device(device)
    model.to(device)    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle_train, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)    
    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)    
    for epoch_i in range(epoch_n):
        epoch_start = datetime.datetime.now()
        print('Эпоха {}'.format(epoch_i))
        h = model.init_hidden(batch_size)        
        model.train()
        mean_train_loss = train_batches_n = 0
        for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
            h = h.data
            if batch_i > max_batches_per_epoch_train:
                break
            batch_x = tools.copy_data_to_device(batch_x, device)
            batch_y = tools.copy_data_to_device(batch_y, device)
            model.zero_grad()
            pred, h = model(batch_x, h)
            #pred = model(batch_x)
            loss = criterion(pred, batch_y)            
            loss.backward()            
            optimizer.step()            
            mean_train_loss += float(loss)
            train_batches_n += 1        
            mean_train_loss /= train_batches_n
        print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n, (datetime.datetime.now() - epoch_start).total_seconds()))
        print('Среднее значение функции потерь на обучении', mean_train_loss)        
        model.eval()
        mean_val_loss = val_batches_n = 0        
        with torch.no_grad():
            for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                if batch_i > max_batches_per_epoch_val:
                    break
                h = model.init_hidden(len(batch_y))
                batch_x = tools.copy_data_to_device(batch_x, device)
                batch_y = tools.copy_data_to_device(batch_y, device)                
                pred, h = model(batch_x, h)
             #   pred = model(batch_x)                
                loss = criterion(pred, batch_y)                
                mean_val_loss += float(loss)
                val_batches_n += 1        
                mean_val_loss /= val_batches_n
        print('Среднее значение функции потерь на валидации', mean_val_loss)        
        if mean_val_loss < best_val_loss:
            best_epoch_i = epoch_i
            best_val_loss = mean_val_loss
            best_model = copy.deepcopy(model)
            print('Новая лучшая модель!')
        elif epoch_i - best_epoch_i > early_stopping_patience:
            print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(early_stopping_patience))
            break        
        lr_scheduler.step(mean_val_loss)
        print()    
    return best_val_loss, best_model

In [ ]:
def predict_with_model(model, dataset, device=None, batch_size=32, return_labels=False, collate_fn=None):
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    results_by_batch = []    
    device = torch.device(device)
    model.to(device)
    model.eval()    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    labels = []
    with torch.no_grad():
        for batch_x, batch_y in dataloader:            
            batch_x = tools.copy_data_to_device(batch_x, device)            
            if return_labels:
                labels.append(batch_y.numpy())
            h = model.init_hidden(len(batch_y))
            batch_pred, h = model(batch_x, h)
            results_by_batch.append(batch_pred.detach().cpu().numpy())    
    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)

In [ ]:
train_pred = predict_with_model(
    best_model,
    list(zip(train.item_name, [classes[i] for i in train['category_id']])),
    collate_fn=collate_batch,
    device=device
)
train_loss = F.cross_entropy(torch.from_numpy(train_pred), torch.from_numpy(np.array(y_train)).long())
print('BCE_train', float(train_loss))
print('ACC_train', f1_score(y_train, train_pred.argmax(-1), average='weighted'))
print()
test_pred = predict_with_model(
    best_model,
    list(zip(test.item_name, [classes[i] for i in test['category_id']])),
    collate_fn=collate_batch,
    device=device
)
test_loss = F.cross_entropy(torch.from_numpy(test_pred), torch.from_numpy(np.array(y_test)).long())
print('BCE_test', float(test_loss))
print('ACC_test', f1_score(y_test, test_pred.argmax(-1), average='weighted'))

## Word2Vec и Fasttext

In [ ]:
import pandas as pd
import os
import random
import gensim

In [ ]:
full_dataset = list(pd.read_csv('drive/MyDrive/Colab Notebooks/nyt-ingredients-snapshot-2015.csv')['input'].dropna())
random.shuffle(full_dataset)

TRAIN_VAL_SPLIT = int(len(full_dataset) * 0.7)
train_source, test_source = full_dataset[:TRAIN_VAL_SPLIT], full_dataset[TRAIN_VAL_SPLIT:]
len(train_source)

In [ ]:
word2vec = gensim.models.Word2Vec(sentences=train_tokenized, size=100,
                                  window=5, min_count=5, workers=4,
                                  sg=1, iter=10)

In [ ]:
# word2vec.wv.most_similar('chicken')

In [ ]:
test_words = ['salad', 'fish', 'salmon', 'sauvignon', 'beef', 'pork', 'steak', 'beer', 'cake', 'coffee', 'sausage', 
              'wine', 'merlot', 'zinfandel', 'trout', 'chardonnay', 'champagne', 'cacao']

In [ ]:
gensim_words = [w for w in test_words if w in word2vec.wv.vocab]
gensim_vectors = np.stack([word2vec.wv[w] for w in gensim_words])

In [ ]:
fasttext = gensim.models.FastText(sentences=train_tokenized, size=100,
                                  window=5, min_count=5, workers=4,
                                  sg=1, iter=10)

In [ ]:
# fasttext.wv.most_similar('chicken')

In [ ]:
ft_words = [w for w in test_words if w in fasttext.wv.vocab]
ft_vectors = np.stack([fasttext.wv[w] for w in ft_words])